In [1]:
import torch
import torchvision as tv

from torch import nn
import time
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

In [2]:
BATCH_SIZE=16

In [ ]:
# Пробуем сложную агументацию данных: горизонтальные повороты, кропы

In [18]:
augs = tv.transforms.Compose([tv.transforms.Resize([64, 64]),
                              tv.transforms.Grayscale(3),
                              tv.transforms.RandomHorizontalFlip(),
                              tv.transforms.RandomResizedCrop(
                              (64, 64), scale=(0.8, 1), ratio=(0.5, 2)),
                              tv.transforms.ToTensor()])

In [19]:
train_dataset = tv.datasets.ImageFolder('/home/plpronchenko/study_data/DL_5_HW/images/images/train/', transform= augs)

In [20]:
test_dataset = tv.datasets.ImageFolder('/home/plpronchenko/study_data/DL_5_HW/images/images/validation/', transform= augs)

In [21]:
pd.DataFrame(train_dataset.targets)[0].value_counts()

3    7164
4    4982
5    4938
2    4103
0    3993
6    3205
1     436
Name: 0, dtype: int64

In [22]:
pd.DataFrame(test_dataset.targets)[0].value_counts()

3    1825
4    1216
5    1139
2    1018
0     960
6     797
1     111
Name: 0, dtype: int64

In [23]:
train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle = True)
test_iter = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle = True)

In [24]:
resnet50 = tv.models.resnet50(pretrained=True)

In [25]:
resnet50.fc = nn.Linear(in_features=2048, out_features=7, bias=True)

In [26]:
model = resnet50

In [27]:
dev = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

In [28]:
model = model.to(dev)

In [29]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [30]:
def train(net, train_iter, test_iter, trainer, num_epochs, dev):
    loss = nn.CrossEntropyLoss(reduction='mean')
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            trainer.zero_grad()
            X, y = X.to(dev), y.to(dev)
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.shape[0]
        test_acc = evaluate_accuracy(test_iter, net, dev)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, '
              'time %.1f sec'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc,
                 time.time() - start))

In [31]:
def evaluate_accuracy(data_iter, net, dev):
    acc_sum, n = torch.Tensor([0]).to(dev), 0
    for X, y in data_iter:
        X, y = X.to(dev), y.to(dev)
        acc_sum += (net(X).argmax(axis=1) == y).sum()
        n += y.shape[0]
    return acc_sum.item() / n

In [32]:
lr, num_epochs = 0.05, 10
trainer = torch.optim.SGD(model.parameters(), lr=lr, momentum = 0.9)
train(model, train_iter, test_iter, trainer, num_epochs, dev)

epoch 1, loss 0.1218, train acc 0.235, test acc 0.260, time 137.7 sec
epoch 2, loss 0.1116, train acc 0.259, test acc 0.285, time 152.6 sec
epoch 3, loss 0.1061, train acc 0.317, test acc 0.358, time 142.8 sec
epoch 4, loss 0.1015, train acc 0.358, test acc 0.377, time 147.7 sec
epoch 5, loss 0.0980, train acc 0.387, test acc 0.409, time 143.7 sec
epoch 6, loss 0.0948, train acc 0.410, test acc 0.410, time 113.1 sec
epoch 7, loss 0.0919, train acc 0.427, test acc 0.434, time 116.6 sec
epoch 8, loss 0.0888, train acc 0.451, test acc 0.447, time 120.6 sec
epoch 9, loss 0.0865, train acc 0.467, test acc 0.471, time 152.4 sec
epoch 10, loss 0.0834, train acc 0.484, test acc 0.489, time 150.1 sec


In [35]:
lr, num_epochs = 0.005, 15
trainer = torch.optim.SGD(model.parameters(), lr=lr, momentum = 0.7)
train(model, train_iter, test_iter, trainer, num_epochs, dev)

epoch 1, loss 0.0676, train acc 0.589, test acc 0.564, time 150.4 sec
epoch 2, loss 0.0668, train acc 0.595, test acc 0.557, time 150.1 sec
epoch 3, loss 0.0662, train acc 0.597, test acc 0.554, time 154.1 sec
epoch 4, loss 0.0657, train acc 0.602, test acc 0.563, time 155.5 sec
epoch 5, loss 0.0654, train acc 0.605, test acc 0.567, time 152.1 sec
epoch 6, loss 0.0645, train acc 0.610, test acc 0.562, time 156.4 sec
epoch 7, loss 0.0639, train acc 0.610, test acc 0.567, time 156.3 sec
epoch 8, loss 0.0636, train acc 0.620, test acc 0.568, time 167.1 sec
epoch 9, loss 0.0631, train acc 0.621, test acc 0.565, time 157.1 sec
epoch 10, loss 0.0625, train acc 0.623, test acc 0.566, time 163.0 sec
epoch 11, loss 0.0620, train acc 0.625, test acc 0.571, time 105.7 sec
epoch 12, loss 0.0614, train acc 0.632, test acc 0.571, time 130.0 sec
epoch 13, loss 0.0607, train acc 0.633, test acc 0.573, time 159.9 sec
epoch 14, loss 0.0603, train acc 0.637, test acc 0.576, time 181.8 sec
epoch 15, loss 

In [ ]:
lr, num_epochs = 0.0001, 25
trainer = torch.optim.AdamW(model.parameters(), lr=lr)
train(model, train_iter, test_iter, trainer, num_epochs, dev)

epoch 1, loss 0.0612, train acc 0.630, test acc 0.562, time 208.6 sec
epoch 2, loss 0.0605, train acc 0.634, test acc 0.570, time 127.4 sec
epoch 3, loss 0.0594, train acc 0.641, test acc 0.578, time 130.6 sec
epoch 4, loss 0.0583, train acc 0.653, test acc 0.577, time 192.5 sec
epoch 5, loss 0.0570, train acc 0.660, test acc 0.575, time 151.4 sec
epoch 6, loss 0.0558, train acc 0.669, test acc 0.577, time 129.7 sec
epoch 7, loss 0.0542, train acc 0.679, test acc 0.581, time 126.9 sec
epoch 8, loss 0.0530, train acc 0.683, test acc 0.573, time 128.6 sec
epoch 9, loss 0.0518, train acc 0.693, test acc 0.578, time 126.8 sec
epoch 10, loss 0.0502, train acc 0.701, test acc 0.579, time 130.1 sec
epoch 11, loss 0.0491, train acc 0.708, test acc 0.577, time 147.8 sec
epoch 12, loss 0.0473, train acc 0.723, test acc 0.583, time 167.5 sec
epoch 13, loss 0.0464, train acc 0.729, test acc 0.586, time 127.3 sec
epoch 14, loss 0.0450, train acc 0.736, test acc 0.591, time 138.1 sec
epoch 15, loss 

In [ ]:
# качество на тесте - около 60%